In [17]:
import pandas as pd
from sklearn.neighbors import KDTree

FILE = './dataset/votes_skills_anon.csv'

df = pd.read_csv(FILE, index_col=0)

df

,SUID,univid,response
0,538.0,5c9e7a6834cda4f3677e662b,Internationality
1,2285.0,5c9e373c34cda4f3677e14ef,Practical relevance / professionalization
2,2285.0,5c9e7d8334cda4f3677e6b4f,Internationality
3,2285.0,5e84b8a01ead1a77be3bd3a8,Soft skills and digital literacy
4,2285.0,5c9e471734cda4f3677e2ed7,Internationality
...,...,...,...
23234,3124.0,5c9e636734cda4f3677e5027,Internationality
23235,3124.0,5c9e7d7a34cda4f3677e6b41,Internationality
23236,3124.0,5c9e5cf434cda4f3677e4593,Internationality
23237,3124.0,5c9e7d6934cda4f3677e6b27,Internationality


### dữ liệu dạng thô: user x vote trường y thuộc lĩnh vực z, trường a thuộc lĩnh vực b ...

### xử lí dữ liệu từ dữ liệu thô sang dạng mỗi trường gồm x votes về mảng này, y votes về mảng kia ... thì mới thành feature của KNN để so sánh tìm xem các trường nào tương đối giống nhau được

In [21]:
# Group by 'univid' and 'response', then count the occurrences of each group
voting = df.groupby(['univid', 'response'])["response"].count().reset_index(name='votes')

voting

,univid,response,votes
0,5c9e345f34cda4f3677e1047,Academic excellence,10
1,5c9e345f34cda4f3677e1047,Internationality,24
2,5c9e345f34cda4f3677e1047,Practical relevance / professionalization,10
3,5c9e345f34cda4f3677e1047,Soft skills and digital literacy,7
4,5c9e345f34cda4f3677e1047,Specialization,18
...,...,...,...
1237,5e8c8c5623d6684db014ea41,Academic excellence,5
1238,5e8c8c5623d6684db014ea41,Internationality,3
1239,5e8c8c5623d6684db014ea41,Practical relevance / professionalization,3
1240,5e8c8c5623d6684db014ea41,Soft skills and digital literacy,3


In [22]:
summary = voting.pivot_table(values='votes', columns='response', index=['univid'])

summary

response,Academic excellence,Internationality,Practical relevance / professionalization,Soft skills and digital literacy,Specialization
univid,,,,,
5c9e345f34cda4f3677e1047,10.0,24.0,10.0,7.0,18.0
5c9e349434cda4f3677e109f,5.0,10.0,12.0,9.0,15.0
5c9e34a834cda4f3677e10bd,8.0,10.0,8.0,6.0,8.0
5c9e34ae34cda4f3677e10c7,1.0,17.0,17.0,14.0,17.0
5c9e34d534cda4f3677e1107,10.0,11.0,15.0,10.0,24.0
...,...,...,...,...,...
5e8c879823d6684db014ea30,8.0,29.0,18.0,23.0,30.0
5e8c89b723d6684db014ea35,6.0,15.0,9.0,16.0,16.0
5e8c8b0f23d6684db014ea37,8.0,28.0,5.0,29.0,24.0


In [35]:
sum_votes_uni = summary.sum(axis=1)

sum_votes_uni

univid
5c9e345f34cda4f3677e1047    1.0
5c9e349434cda4f3677e109f    1.0
5c9e34a834cda4f3677e10bd    1.0
5c9e34ae34cda4f3677e10c7    1.0
5c9e34d534cda4f3677e1107    1.0
                           ... 
5e8c879823d6684db014ea30    1.0
5e8c89b723d6684db014ea35    1.0
5e8c8b0f23d6684db014ea37    1.0
5e8c8b7523d6684db014ea3b    1.0
5e8c8c5623d6684db014ea41    1.0
Length: 250, dtype: float64

In [41]:
summary = summary.div(sum_votes_uni, axis=0)
summary = summary.fillna(0)
summary

response,Academic excellence,Internationality,Practical relevance / professionalization,Soft skills and digital literacy,Specialization
univid,,,,,
5c9e345f34cda4f3677e1047,0.144928,0.347826,0.144928,0.101449,0.260870
5c9e349434cda4f3677e109f,0.098039,0.196078,0.235294,0.176471,0.294118
5c9e34a834cda4f3677e10bd,0.200000,0.250000,0.200000,0.150000,0.200000
5c9e34ae34cda4f3677e10c7,0.015152,0.257576,0.257576,0.212121,0.257576
5c9e34d534cda4f3677e1107,0.142857,0.157143,0.214286,0.142857,0.342857
...,...,...,...,...,...
5e8c879823d6684db014ea30,0.074074,0.268519,0.166667,0.212963,0.277778
5e8c89b723d6684db014ea35,0.096774,0.241935,0.145161,0.258065,0.258065
5e8c8b0f23d6684db014ea37,0.085106,0.297872,0.053191,0.308511,0.255319


In [47]:
kd_tree = KDTree(summary)
distance, index = kd_tree.query(summary[9:10], k=5)

index = index.tolist()[0]

index
summary.iloc[index]

response,Academic excellence,Internationality,Practical relevance / professionalization,Soft skills and digital literacy,Specialization
univid,,,,,
5c9e35b234cda4f3677e126b,0.047059,0.258824,0.152941,0.247059,0.294118
5c9e5b5e34cda4f3677e42fb,0.078125,0.270833,0.135417,0.229167,0.286458
5c9e3fc034cda4f3677e22e3,0.087500,0.250000,0.150000,0.225000,0.287500
5e8c879823d6684db014ea30,0.074074,0.268519,0.166667,0.212963,0.277778
5c9e7c4034cda4f3677e6937,0.075472,0.264151,0.169811,0.207547,0.283019


### kết quả tìm được 5 trường giống với row 9  nhất ở bảng trên.